<a href="https://colab.research.google.com/github/SolNam-UI/CTD/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import 하는 곳

In [ ]:
!pip install quickdraw tensorflow ColabTurtle Pillow

  Preparing metadata (setup.py) ... done
  Created wheel for ColabTurtle: filename=ColabTurtle-2.1.0-py3-none-any.whl size=7643 sha256=27e4a3f6e0201c5ec3bf266039dd78b600e07a6cf610e46b320bbc5b607208ab
  Stored in directory: /root/.cache/pip/wheels/f6/9e/81/137e7da25129474562d30f8660be599e5c8d79228cb747e5b9
Successfully built ColabTurtle


In [ ]:
import numpy as np
import random
from PIL import Image
from quickdraw import QuickDrawDataGroup, QuickDrawData
from keras import layers
from keras import models
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from ColabTurtle import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/파이썬 프로젝트 CTD"
!git clone https://github.com/SolNam-UI/CTD.git
!git clone https://gist.github.com/8409b3feec20f159d8a50b0a811d3bca.git #Colab 환경에서 사용자가 그릴 수 있게 해주는 오픈소스
%run 8409b3feec20f159d8a50b0a811d3bca/draw.py
%cd "/content/drive/Mydrive/파이썬 프로젝트 CTD/storage"

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

Turtle로 사용자가 그림 그리게 하기 (동렬)

In [5]:
import numpy as np
import random
from PIL import Image, ImageDraw
import io
import os
import time
import base64 # 이미지 인코딩/디코딩용
from IPython.display import display, HTML, Javascript
from google.colab import drive
import ColabTurtle.Turtle as turtle

drive.mount('/content/drive')
%cd "/content/drive/MyDrive/파이썬 프로젝트 CTD"
!git clone https://github.com/SolNam-UI/CTD.git
!git clone https://gist.github.com/8409b3feec20f159d8a50b0a811d3bca.github
%run 8409b3feec20f159d8a50b0a811d3bca/draw.py

SAVE_PATH = "/content/sample_data/"
os.makedirs(SAVE_PATH, exist_ok=True)

def run_drawing_and_save():
    print("------------------------------------------------------------------")
    print("              Colab 마우스 드로잉 인터페이스가 로드되었습니다.       ")
    print("              아래에 나타나는 캔버스에 마우스로 그림을 그려주세요.  ")
    print("              그림을 다 그리신 후에는 '완료' 버튼을 클릭하거나,     ")
    print("              새로운 셀에서 'get_drawing()'을 호출하여 그림을 받아오세요.")
    print("------------------------------------------------------------------")

    print("\n[안내]: 그림 그리기 후 다음 셀에서 'get_drawing()'을 실행하여 그림 데이터를 가져와 주세요.")
    print("       그 후, 가져온 데이터를 변수에 할당하여 아래 저장 로직을 이어서 실행할 수 있습니다.")

def save_drawing_to_gif(drawing_data):
    """
    Colab의 draw.py를 통해 얻은 그림 데이터를 GIF 파일로 저장하
    """
    if not drawing_data:
        print("그림 데이터가 없습니다. 그림을 먼저 그려주세요.")
        return

    print("\nColabTurtle을 통해 그림을 시각화합니다...")
    turtle.clearscreen()
    turtle.setup(width=800, height=600)
    turtle.bgcolor("white")
    turtle.showborder()
    turtle.speed(0)

    drawing_turtle = turtle.Turtle()
    drawing_turtle.penup()
    drawing_turtle.pencolor("black")
    drawing_turtle.pensize(2)

    CANVAS_WIDTH = 400
    CANVAS_HEIGHT = 400
    TURTLE_OFFSET_X = -CANVAS_WIDTH / 2
    TURTLE_OFFSET_Y = CANVAS_HEIGHT / 2

    for stroke in drawing_data:
        is_first_point_in_stroke = True
        for x, y, _ in stroke:
            turtle_x = x + TURTLE_OFFSET_X
            turtle_y = TURTLE_OFFSET_Y - y

            if is_first_point_in_stroke:
                drawing_turtle.penup()
                drawing_turtle.goto(turtle_x, turtle_y)
                drawing_turtle.pendown()
                is_first_point_in_stroke = False
            else:
                drawing_turtle.goto(turtle_x, turtle_y)

    img_width, img_height = 400, 400
    image = Image.new("RGB", (img_width, img_height), "white")
    draw_pil = ImageDraw.Draw(image)

    line_color = "black"
    line_width = 2 # 픽셀 단위 선 굵기

    for stroke in drawing_data:
        if len(stroke) < 2: # 점이 하나뿐인 스트로크는 건너뛰기
            continue

        # 스트로크 내의 점들을 연결하여 선을 그립니다.
        for i in range(len(stroke) - 1):
            x1, y1, _ = stroke[i]
            x2, y2, _ = stroke[i+1]
            draw_pil.line([(x1, y1), (x2, y2)], fill=line_color, width=line_width)

    # 3. PIL 이미지를 임시 파일로 저장하고 ImageMagick으로 GIF 변환
    temp_png_file = os.path.join(SAVE_PATH, "temp_drawing_pil.png")
    final_gif_file = os.path.join(SAVE_PATH, "user_drawing.gif")

    try:
        image.save(temp_png_file, "PNG")
        print(f"\n그려진 그림이 임시 PNG 파일로 저장되었습니다: {temp_png_file}")

        convert_command = f"convert {temp_png_file} {final_gif_file}"
        os.system(convert_command)

        if os.path.exists(final_gif_file):
            print(f"그림이 GIF 파일로 성공적으로 저장되었습니다: {final_gif_file}")
            print(f"파일 경로: {final_gif_file}")

        else:
            print(f"GIF 파일 변환에 실패했습니다. ImageMagick이 올바르게 설치되었는지 확인해주세요.")
            print(f"변환 명령어: {convert_command}")

        os.remove(temp_png_file) # 임시 PNG 파일 삭제
        print(f"임시 PNG 파일 {temp_png_file}이 삭제되었습니다.")

    except Exception as e:
        print(f"그림을 저장하고 변환하는 중에 오류가 발생했습니다: {e}")
        print("ImageMagick이 설치되어 있고 파일 경로가 올바른지 확인해주세요.")

# ---------- 실행 가이드 ----------
# 1. 위 코드를 전체 실행합니다. (셀 실행)
# 2. 출력된 캔버스에 마우스로 그림을 그립니다.
# 3. 그림을 다 그린 후, **새로운 코드 셀**을 추가하여 다음을 실행합니다.
#    drawing_data_from_canvas = get_drawing()
#    save_drawing_to_gif(drawing_data_from_canvas)
#    이렇게 하면, get_drawing()으로 그림 데이터를 가져와서 save_drawing_to_gif 함수에 전달하고 저장합니다.
# -------------------------------

# 스크립트 시작 시 드로잉 인터페이스 로드 안내
if __name__ == "__main__":
    run_drawing_and_save()



Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/파이썬 프로젝트 CTD'
/content
Cloning into 'CTD'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 30 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 18.06 KiB | 9.03 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Cloning into '8409b3feec20f159d8a50b0a811d3bca.github'...
fatal: could not read Username for 'https://gist.github.com': No such device or address


Exception: File `'8409b3feec20f159d8a50b0a811d3bca/draw.py'` not found.

In [ ]:
# 그림 그리기
def inputDrawing():
    draw(filename="dd.png", w=280, h=280, line_width=10)

    # 사용자가 그린 이미지 전처리
    img = Image.open("/content/dd.png").resize((28, 28)).convert("L")
    imgTemp = np.array(img) / 255.0
    inData = imgTemp.reshape(1, 28, 28, 1)

    predict(inData)

def predict(inData):
    # 예측
    prediction = model.predict(inData)
    result = trainList[np.argmax(prediction)]
    print("정답은: ", result + "입니다.")
    if (result == goalImg) :
        print("성공!")
    else:
        print("실패 다시 그리세요!")
        inputDrawing()

# 모델 정의
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation = "relu", input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(units = 64, activation = "relu"))
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# 이미지 리스트 로드
trainList = QuickDrawData().drawing_names[:50]

# 이미지 학습
trainImg, trainLabel = [], []
for i, label in enumerate(trainList):
    drawings = list(QuickDrawDataGroup(label).drawings)[:100]
    for d in drawings:
        img = d.image.resize((28, 28)).convert("L")
        img_array = np.array(img) / 255.0
        trainImg.append(img_array)
        trainLabel.append(i)

trainImg = np.array(trainImg)
trainImg = trainImg.reshape(-1, 28, 28, 1)

trainLabel = np.array(trainLabel)

# 모델 학습
model.fit(trainImg, trainLabel, epochs=2, batch_size=32)

# 이미지 리스트에서 그릴 이미지 할당
goalImg = trainList[int(random.randrange(0,49))]
print("목표 그림: ", goalImg)

inputDrawing()

axe
loading aircraft carrier drawings
load complete
loading airplane drawings
load complete
loading alarm clock drawings
load complete
loading ambulance drawings
load complete
loading angel drawings
load complete
loading animal migration drawings
load complete
loading ant drawings
load complete
loading anvil drawings
load complete
loading apple drawings
load complete
loading arm drawings
load complete
loading asparagus drawings
load complete
loading axe drawings
load complete
loading backpack drawings
load complete
loading banana drawings
load complete
loading bandage drawings
load complete
loading barn drawings
load complete
loading baseball bat drawings
load complete
loading baseball drawings
load complete
loading basket drawings
load complete
loading basketball drawings
load complete
loading bat drawings
load complete
loading bathtub drawings
load complete
loading beach drawings
load complete
loading bear drawings
load complete
loading beard drawings
load complete
loading bed drawin

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
정답은:  beard입니다.
실패 다시 그리세요!


AttributeError: 'NoneType' object has no attribute 'split'

Turtle로 출력하기 (동렬)

ModuleNotFoundError: No module named 'turtle2img'